<a href="https://colab.research.google.com/github/JavierPachas/llama3/blob/main/llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

In [3]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
messages = [
    {"role": "system", "content": "Eres un agente muy eficiente y respondes en español a menos que se te indique lo contrario"},
    {"role": "user", "content": """Haz un resumen en 80 palabras del siguiente texto:
                                ¿Qué son y para qué se utilizan los productos fitosanitarios? Los productos fitosanitarios (insecticidas, fungicidas, herbicidas…) son mezclas químicas cuyo objetivo es proteger a los vegetales de organismos nocivos. Contribuyen a aumentar los rendimientos en la agricultura, pero al mismo tiempo su uso incorrecto puede entrañar riesgos para la salud humana y el medioambiente. De acuerdo con Eurostat,  la venta de productos fitosanitarios en la UE-28 supuso 2016 372.567 toneladas 2017  2,7 % 362.626 toneladas El 19,8 % de las ventas se realizó en España, siendo el país con mayor venta de este tipo de productos (Perfil Ambiental de España 2018). ¿Cómo llega al medio ambiente? La presencia de sustancias químicas nocivas en los diferentes  compartimentos ambientales es una preocupación en aumento.  Muchas de esas sustancias provienen de la aplicación de productos fitosanitarios. Registro Oficial de Productos y Material Fitosanitario (ROPMF) 1. Si no eres usuario profesional, debes usar sólo aquellos productos destinados al público en general. 2. Antes de usar cualquier producto, lee detenidamente la etiqueta. Úsalo de acuerdo a las condiciones de utilización: cultivo y plaga objetivo, dosis, momento de aplicación, etc. Emplea las medidas de protección individual. Para poder comercializarse, estos productos deben tener una autorización y estar inscritos en el Registro Oficial de Productos Fitosanitarios del Ministerio de Agricultura, Pesca y Alimentación. La autorización se obtiene tras una exhaustiva evaluación de sus riegos. Es necesario que los productos fitosanitarios, para el uso al que se autoricen, sean seguros para la salud humana, no tengan efectos negativos en el medio ambiente y los residuos que permanecen en las partes comestibles de los vegetales se encuentren dentro de los umbrales aceptables. ¿Qué hay que tener en cuenta  a la hora de aplicar el fitosanitario? 3. Fíjate en el apartado de Mitigación de Riesgos Medioambientales. Cuantas menos frases de tipo “SPe” aparezcan, menor riesgo ambiental. 4. Los envases son considerados residuos peligrosos. Deséchalos en los puntos habilitados para ello siguiendo las instrucciones especificadas en el etiquetado. Prioriza los métodos alternativos no químicos,  siempre que sea posible • Aplicación de las Guías de Gestión Integrada de Plagas • Limpieza periódica de la maquinaria de propágulos nocivos • Uso de prácticas de fertilización, riego y drenaje equilibradas • Aumenta la diversidad en la rotación de cultivos • Uso de simiente y plantones certificados y de variedades resistentes o tolerantes • Mantenimiento de franjas de vegetación natural permanente • Si procede, desbroce previo a la aplicación de un herbicida para aumentar la eficacia y reducir la cantidad necesaria Toma las medidas necesarias para prevenir la contaminación, y reducir las aplicaciones Usa productos de bajo riesgo en zonas sensibles (de extracción de agua para consumo humano y  zonas de protección de hábitats y especies). ¿Cómo evitar la contaminación de las aguas? 5 metros Respeta una banda de seguridad de 5 metros como mínimo con respecto a las masas de agua superficial, o en superficies muy Cubre los puntos de agua, como pozos, de las parcelas a tratar, para evitar la contaminación puntual, durante el tratamiento. 50 metros Deja sin tratar una distancia de 50 metros como mínimo respecto a los puntos de extracción de agua para consumo humano. permeables. en su caso, lo indicado en la etiqueta. Prioriza productos que resulten menos peligrosos para los organismos acuáticos. Si llevan el pictograma y la palabra de advertencia Atención, son más tóxicos. Fíjate en la indicación de peligro. La toxicidad disminuye según el siguiente orden: H400 y H4 H400 y H4 H4 H4 H4 H4 H4 S H4 Está en tu mano contribuir a su reducción H400 y H4 H4 S MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL RETO DEMOGRÁFICO GOBIERNO DE ESPAÑA H4 H4 S
                                """},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

En 80 palabras:

Los productos fitosanitarios, como insecticidas, fungicidas y herbicidas, se utilizan para proteger cultivos de plagas y enfermedades. Aunque aumentan los rendimientos agrícolas, su uso incorrecto puede afectar la salud humana y el medio ambiente. Es importante leer la etiqueta y seguir las instrucciones para evitar la contaminación y reducir los riesgos. Es fundamental priorizar métodos alternativos no químicos y productos de bajo riesgo en zonas sensibles.
